In [ ]:
!pip install datasets -qq

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
from tqdm import tqdm

from transformers import AutoModel, AutoTokenizer
import datasets
from datasets import load_dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

датасеты можете брать отсюда: https://huggingface.co/datasets \
выбираете датасет и вставляете в load_dataset() его название \
есть параметр name (название датасета, если их несколько) \
и параметр split (train, test и validation) - если вам нужен только train, test или validation

In [ ]:
dataset = load_dataset('cardiffnlp/tweet_eval', name='irony')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [ ]:
dataset

модельки брать можете здесь: https://huggingface.co/models \
выбираем подходящую и копируем название

In [ ]:
# загружаем модель (bert-base-uncased -> uncased - значит ей неважно, какой регистр (просто .lower() делается): эмбеддинги 'привет' и 'Привет' в одинаковых контекстах будут одинаковы)
# и токенизатор (у берта токенизатор - WordPiece, можно почитать про него тут: https://huggingface.co/learn/llm-course/ru/chapter6/6)
model_name = 'google-bert/bert-base-uncased'

model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
# проверим максимальную длину в токенах для всех датасетов (чтобы понять, до какой длины делать padding и truncation)

max_length_train = 0
for example in dataset['train']:
    max_length_train = max(max_length_train, (len(tokenizer(example['text'])['input_ids'])))

max_length_test = 0
for example in dataset['test']:
    max_length_test = max(max_length_test, (len(tokenizer(example['text'])['input_ids'])))

max_length_validation = 0
for example in dataset['validation']:
    max_length_validation = max(max_length_validation, (len(tokenizer(example['text'])['input_ids'])))

max_length_train, max_length_test, max_length_validation

In [ ]:
# Проверим количество параметров у нашей модели (bert-base-uncased)
print(sum([p.numel() for p in model.parameters()])) # -> # всех параметров
sum([p.numel() for p in model.parameters() if p.requires_grad]) # -> # параметров, которые будут обучаться

In [ ]:
# функция препроцессинга текстов
# max_length=328 просто потому что максимальная длина немного меньше этого значения (328 взял из головы с небольшим запасом)
def preproc(examples):
    input_ids = tokenizer(examples['text'], max_length=328, padding='max_length', truncation=True)
    labels = examples['label']
    input_ids['label'] = labels
    return input_ids

In [ ]:
# применяем функцию препроцессинга (токенизируем текст)
dataset = dataset.map(preproc)

In [ ]:
# чтобы все столбцы, которые могут, стали тензорами (мы будем работать именно с тензорами)
dataset.set_format('torch')

In [ ]:
# создаем даталоадеры (они формируют батчи, чтобы мы потом по ним итерировались и подавали в модель сразу батч данных)
train_loader = torch.utils.data.DataLoader(dataset['train'], batch_size=8, shuffle=True) # для трейна перемешиваем данные [shuffle=True] (чтобы избежать структурированных данных в одном батче (для разнообразия и стохастичности))
test_loader = torch.utils.data.DataLoader(dataset['test'], batch_size=8, shuffle=False) # нет смысла шафлить данные для теста и валидации
val_loader = torch.utils.data.DataLoader(dataset['validation'], batch_size=8, shuffle=False)

In [ ]:
# посмотрим размерности
for batch in train_loader:
    print(batch['input_ids'].shape) # .shape -> [batch_size, seq_len]
    print(batch['attention_mask'].shape) # .shape -> [batch_size, seq_len]
    print(batch['label'].shape) # .shape -> [batch_size]
    break

torch.Size([8, 328])
torch.Size([8, 328])
torch.Size([8])


In [ ]:
# посмотрим размерность выхода модели (на вход подаем 'input_ids')
model.eval()
model(batch['input_ids']).last_hidden_state.shape

torch.Size([8, 328, 768])

In [ ]:
# посмотрим размерность эмбеддингов в конфиге нашей модели (для того, чтобы узнать размерность, которая будет на входе в голову классификации)
model.config.hidden_size

768

In [ ]:
# для примера заморозим все слои берта
# а будем обучать только голову классификации
for param in model.parameters():
    param.requires_grad = False

*   В берте в начало каждой последовательности токенов добавляется специальный токен - [CLS], который аккумулирует в себе информацию о всей последовательности, мы и будем использовать этот токен (просто подавать [CLS] токен (размерности -> [batch_size, emb_size]) на вход в голову классификации)
*   Можно также просто брать выход модели (размерности [batch_size, seq_len, emb_size]) и усреднять по размерности seq_len



In [ ]:
class CLF(nn.Module):
    def __init__(self, base_model, num_classes=2):
        super(CLF, self).__init__()
        self.base_model = base_model
        # голова классификации:
        self.lin1 = nn.Linear(base_model.config.hidden_size, 256)
        self.relu = nn.LeakyReLU()
        self.lin2 = nn.Linear(256, num_classes)

    def forward(self, input_ids, attn_mask):
        out = self.base_model(input_ids=input_ids, attention_mask=attn_mask).last_hidden_state # out.shape -> [batch_size, seq_len, emb_size]
        # берем CLS токен:
        cls = out[:, 0] # cls.shape -> [batch_size, emb_size]
        return self.lin2(self.relu(self.lin1(cls))) # .shape -> [batch_size, num_classes]

In [ ]:
model = CLF(base_model=model, num_classes=2).to(device)

In [ ]:
# проверим сколько параметров мы будем обучать
# в нашем случае - это просто # параметров головы классификации
sum([p.numel() for p in model.parameters() if p.requires_grad])

# их можно посчитать руками (#параметров двух линейных слоев):
# base_model.config.hidden_size (768) * 256 + 256 (bias)
# 256 * num_classes (2) + 2 (bias)

197378

In [ ]:
# так как мы обучаем только голову классификацию (параметры самого берта заморожены), то lr=5e-4
# если хотите обучать еще и берта (разморозить все параметры), то лучше поставить поменьше, нап, lr=5e-5
optim = torch.optim.AdamW(model.parameters(), lr=5e-4)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# прописываем классический трейн пайплайн

epochs = 3
for epoch in range(epochs):
    model.train()
    train_running_loss = 0.0
    for batch in tqdm(train_loader, desc=f'Training, epoch: {epoch}/{epochs}'):
        input_ids = batch['input_ids'].to(device)
        attn_mask = batch['attention_mask'].to(device)
        label = batch['label'].to(device)

        logits = model(input_ids, attn_mask)
        loss = loss_fn(logits, label)

        train_running_loss += loss.item()

        optim.zero_grad()
        loss.backward()
        optim.step()

    train_loss = train_running_loss / len(train_loader)
    print(f'Train, Epoch: {epoch}, train_loss: {train_loss:.4f}')

    model.eval()
    eval_running_loss = 0.0
    for batch in tqdm(val_loader, desc=f'Validation, epoch: {epoch}/{epochs}'):
        input_ids = batch['input_ids'].to(device)
        attn_mask = batch['attention_mask'].to(device)
        label = batch['label'].to(device)

        with torch.no_grad():
            logits = model(input_ids, attn_mask)
            loss = loss_fn(logits, label)
            eval_running_loss += loss.item()

    eval_loss = eval_running_loss / len(val_loader)
    print(f'Validation, Epoch: {epoch}, val_loss: {eval_loss:.4f}')

In [ ]:
# теперь посмотрим метрики (через classification_report) на test_loader'е
model.eval()

all_preds = []
all_labels = []
for batch in tqdm(test_loader, desc=f'Testing'):
    input_ids = batch['input_ids'].to(device)
    attn_mask = batch['attention_mask'].to(device)
    label = batch['label'].to(device)

    with torch.no_grad():
        logits = model(input_ids, attn_mask)
        preds = torch.argmax(logits, dim=-1) # берем класс с большим числом (логитом)

    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(label.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

report = classification_report(all_labels, all_preds, target_names=['class 0', 'class 1'], digits=4)

Testing: 100%|██████████| 98/98 [00:16<00:00,  5.88it/s]


In [ ]:
print(report)

              precision    recall  f1-score   support

     class 0     0.8444    0.4820    0.6137       473
     class 1     0.5233    0.8650    0.6521       311

    accuracy                         0.6339       784
   macro avg     0.6839    0.6735    0.6329       784
weighted avg     0.7171    0.6339    0.6290       784



## ДЗ:  

1.   Попробовать другие датасеты помимо этого (например, просто поменять name), но не забывайте про дисбаланс классов (чтобы модель не выучила только один класс)
2.   Переписать трейн пайплайн на transformers.Trainer (вот документация: https://huggingface.co/docs/transformers/main_classes/trainer)
3.   Выбить адекватное качество (поперебирать параметры, попробовать другие модели, например: https://huggingface.co/Tochka-AI/ruRoPEBert-e5-base-2k, поработать с данными (в некоторых датасетах нужна доп обработка руками))